# La Bibliografía Española de Cartografía

Este notebook utiliza la Bibliografía Española de Cartografía es una publicación cuyo objetivo principal es dar a conocer el material cartográfico publicado en España, que ingresa en la Biblioteca Nacional de España que incluye mapas, planos, cartas náuticas, atlas, etc., tanto en formato impreso como electrónico. 

En el año 2007 se inició la publicación en línea y desde el 2010 se incorporan a esta bibliografía los atlas, antes recogidos en la Bibliografía Española de Monografías. Se publica anualmente y es consultable en línea.

https://datos.gob.es/es/catalogo/e00123904-bibliografia-espanola-de-cartografia-2017

### Importando las librerías de código

In [38]:
# https://pypi.org/project/pymarc/
import pymarc, re, csv
import pandas as pd
from pymarc import parse_xml_to_array
from datapackage import Package

### Generando un fichero CSV como salida con el contenido procesado

In [44]:
csv_salida = csv.writer(open('registros_marc_bne.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_salida.writerow(['titulo', 'autor', 'variante_titulo', 'extension', 'distribuidor', 'materias', 'nota', 'detalles'])

76

### Leyendo los archivos originales

In [46]:
registros = parse_xml_to_array(open('BNE-cartografia/BN_CARTOGRAFIA_2017-MARCXML.xml'))

for registro in registros:
    
    titulo = autor = variante_titulo = extension = distribuidor = materias = nota = detalles =''
    
    # titulo
    if registro['245'] is not None:
      titulo = registro['245']['a']
      if registro['245']['b'] is not None:
        titulo = titulo + " " + registro['245']['b']
    
    # autor
    if registro['100'] is not None:
      autor = registro['100']['a']
    elif registro['110'] is not None:
      autor = registro['110']['a']
    elif registro['700'] is not None:
      autor = registro['700']['a']
    elif registro['710'] is not None:
      autor = registro['710']['a']
    
    # variante de titulo
    if registro['246'] is not None:
      variante_titulo = registro['246']['a']
    
    
    # Physical Description - extent
    for f in registro.get_fields('300'):
        extension = f.get_subfields('a')
        if len(extension):
            extension = extension[0]
            # TODO cleaning
        detalles = f.get_subfields('b')
        if len(detalles):
            detalles = detalles[0]
            
    # distribuidor
    if registro['260'] is not None:
      distribuidor = registro['260']['b']
    
    # nota
    if registro['501'] is not None:
      nota = registro['501']['a']
    
    # materia
    if registro['650'] is not None:
        materias = '' 
        for f in registro.get_fields('650'):
            materias += f.get_subfields('a')[0] + ' -- '
            materias += f.get_subfields('v')[0] + ' -- '
            
        materias = re.sub(' -- $', '', materias)
    
    csv_salida.writerow([titulo,autor,variante_titulo,extension,distribuidor,materias,nota,detalles])

## Leyendo el fichero CSV 

In [47]:
# Este comando añade el contenido del fichero a un Pandas DataFrame
df = pd.read_csv('registros_marc_bne.csv')

## Consultando el contenido

In [48]:
df

,titulo,autor,variante_titulo,extension,distribuidor,materias,nota,detalles
0,14 routen um Manacor,Manacor,Vierzehn routen um Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas -- Manacor,Incluye fotografías,col.
1,14 routes in Manacor,Manacor,Fourteen routes in Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas -- Manacor,Incluye fotografías,col.
2,14 rutas por Manacor,Manacor,Catorce rutas por Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas -- Manacor,Incluye fotografías,col.
3,14 rutes per Manacor,Manacor,Quatorze rutes per Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas -- Manacor,Incluye fotografías,col.
4,A Coruña,Turismo de Galicia,NaN,1 plano,Turismo de Galicia,NaN,Al verso: Información y localización en el pla...,col.
...,...,...,...,...,...,...,...,...
1672,Estella - Lizarra,"Cobo, Ana",NaN,1 plano,Office du Tourisme de Lekunberri,NaN,"Inserta: Información de restaurantes, alojamie...",col.
1673,Estella-Lizarra,"Cobo, Ana",NaN,1 plano,Estella Tourist Office,NaN,"Inserta: Información de restaurantes, alojamie...",col.
1674,Estepona,Instituto Geográfico Nacional (España),NaN,1 h.,Centro Nacional de Información Geográfica,NaN,NaN,col.
1675,Europa física,Instituto Cartográfico Latino,NaN,1 mapa,Vicens Vives,NaN,Al verso: Europa política,col.


### Consultando las columnas

In [49]:
df.columns

Index(['titulo', 'autor', 'variante_titulo', 'extension', 'distribuidor',
       'materias', 'nota', 'detalles'],
      dtype='object')

### ¿Cuántos registros existen?

In [50]:
len(df)

1677

## Explorando las materias
### Creamos una lista de materias y la ordenamos alfabéticamente

In [51]:
df['materias'][2]

'Excursionismo -- Mapas -- Manacor'

In [52]:
df['materias'].str.split(' -- ', expand=True).stack()

0     0    Excursionismo
      1            Mapas
      2          Manacor
1     0    Excursionismo
      1            Mapas
               ...      
1667  4            Mapas
      5         Portugal
1676  0       Hidrología
      1            Mapas
      2       País Vasco
Length: 1731, dtype: object

In [53]:
# Obtener valores únicos
materias = pd.unique(df['materias'].str.split(' -- ', expand=True).stack()).tolist()
for materia in sorted(materias, key=str.lower):
    print(materia)

Aceite de oliva
Actividades recreativas al aire libre
Alaior
Albergues juveniles
Alcanar
Algarve
Alicante (Provincia)
Almería (Provincia)
Alojamientos turísticos
Alt Empordà
Andalucía
Aneto (España, Pico)
Antzuola
Aralar (España, Sierra)
Armiñón
Arquitectura
Arquitectura modernista
Arquitectura románica
Arte románico
Artesa de Segre
Astorga
Asturias (Comunidad Autónoma)
Atzeneta del Maestrat
Aves
Bagà
Balaitús (Macizo)
Banyuls-sur-Mer
Barcelona
Barcelona (Provincia)
Bardenas Reales (Navarra, Comarca)
Bares
Barranca (Navarra, Comarca)
Bellcaire d'Empordà
Benabarre
Benavente
Benicarló
Besalú
Bidasoa (Valle)
Bilbao
Bodegas
Boumort (España, Sierra)
Buceo
Cabrils
Cabuérniga
Cadaqués
Calonge
Camariñas
Cambrils
Caminos
Cantabria (Comunidad Autónoma)
Cardedeu
Carreras campo a través (Atletismo)
Carreras ciclistas
Carreras de automóviles
Carreteras
Cataluña
Centrales hidroeléctricas
Centros docentes
Cercs
Cerdanya
Cervelló
Ciclismo
Ciclismo todoterreno
Cicloturismo
Cine
Cines
Comarcas
Comercios

### También podemos calcular con qué frecuencia se usa una materia

In [54]:
# Partir las materias y obterner el número de ocurrencias
materia_contador = df['materias'].str.split(' -- ').apply(lambda x: pd.Series(x).value_counts()).sum().astype('int').sort_values(ascending=False).to_frame().reset_index(level=0)
# Añadimos las columnas
materia_contador.columns = ['materia', 'contador']
# Mostrar con barras horizontales
display(materia_contador.style.bar(subset=['contador'], color='#d65f5f').set_properties(subset=['contador'], **{'width': '300px'}))

,materia,contador
0,Mapas,664
1,Excursionismo,169
2,Carreteras,132
3,Senderismo,107
4,Planos,107
5,Comercios,100
6,Geología,28
7,Nombres geográficos,28
8,Cicloturismo,23
9,Cataluña,15
